<a href="https://colab.research.google.com/github/ccwbroomfield/NLP_work/blob/main/BytePairEncoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets
from datasets import load_dataset
dataset_train = load_dataset('imdb', split="train")
dataset_test = load_dataset('imdb', split="test")

In [ ]:
from typing import List
from collections import Counter
def learn(text: List[str], k: int):
  print("learn method")
  words = ""
  
  for i in range(len(text)):      #combine text entries, replace spaces with _
    words += text[i] + " "

  word_dict = words.split(" ")

  count_dict = Counter()  #the number of times each word appears in words
  for i in word_dict:
    count_dict[tuple(list(i + "_"))] += 1   #count occurences of word, store as list of chars with "_" at end
                                            #storing as tuple so i can use like list, but hashable

  words = words.replace(" ", "_")
  alphabet = list(words)
  alphabet = set(alphabet)  #get distinct chars
  
  count = Counter()    #the number of times each character appears next to another

  for word in list(count_dict):         #get initial count of tokens
    for c in range(len(word) - 1):
      if "_" not in word[c]:
        count[word[c] + word[c+1]] += count_dict[word]   #not just 1 since this word appears count_dict[word] number of times


  disc_tokens = [] # keep track of all new tokens after original alphabet
  for i in range(k):   #main loop to get k tokens
    token = ""

    j = 1
    while True:  #get the most common token that hasnt been used already
      if (count.most_common(j)[j-1][0] in disc_tokens):
        j+=1
      else:
        token = count.most_common(j)[j-1][0]
        break
    
    alphabet.add(token)
    disc_tokens.append(token)  #add the token to discovered tokens and alphabet


    new_count_dict = Counter()
    for word in list(count_dict):            #go through every word in the corpus
      new_word = []
      k=0
      while k < len(word):             #go through every character, replacing pairs of tokens with new token
        if k < len(word) - 1 and token == word[k] + word[k+1]:
          new_word.append(token)
          if k <len(word) - 2 and "_" not in token:    #if there's space, get the count of the next possible token and add it to count. doesn't change dict
            count[word[k] + word[k + 1] + words[k + 2]] += count_dict[word]
          if k > 0 and "_" not in word[k-1]: #need to look for tokens where thing before is being merged with our token too
            count[word[k-1] + word[k] + words[k+1]] += count_dict[word]
          k += 1
        else:    #if not the token just add it back and don't skip index
          new_word.append(word[k])
        k += 1
      new_count_dict[tuple(new_word)] = count_dict[word]   #after while loop, add the new word to new count dict with num occurrences

    count_dict = new_count_dict    #after for loop, update our count_dict so it has all the new words with tokens replaced

  return disc_tokens    #return the k tokens found. Could also return alphabet here if preferred

  pass

In [ ]:
from typing import List
def segment(text: str, tokens: List[str]):
  print("segment method")

  words = text
  words = words.replace(" ", "_")
  words = list(words)

  for i in tokens:
    new_text = []
    j = 0
    while j < len(words):
      if j < len(words) - 1 and (i == words[j] + words[j + 1]):
        new_text.append(i)
        j += 1
      else:
        new_text.append(words[j])
      j += 1
    words = new_text
  return words
  pass

Heap's/Herdan's law predicts that the number of distinct words in a corpus of text will grow as Kn^Β, where K is generall between 10-100 and B is between .4 and .6 for English collections. n is the number of words in the text. For the imdb datasets, n is 32,369,810. 
This means that we might expect the number of distinct words in the corpus to be around 50 * 32,369,810^.5 = 284,472. On the low end we might expect 10 * 32,369,810^.4 = 10,093. 

> The first 30 new tokens found were 'e_', 's_', 'th', 'he', 't_', 'in', 'd_', 'er', 'an', 'n_', 'r_', 're', 'y_', 'is', 'the_', ',_', 'nd', 'en', 'on', 'at', '._', 'o_', 'ng', 'or', 'es', 'it', 'ha', 'to', 'st', 'ou'. After significant optimization, this computation took 59s to complete. To find all ~285,000 would take considerable time, and exceed the colab limit. 






In [ ]:
import nltk
from nltk.tokenize import word_tokenize, wordpunct_tokenize, WhitespaceTokenizer
nltk.download('punkt')
words = ""
text = dataset_test["text"]
for i in range(len(text)):      #combine text entries, replace spaces with _
    words += text[i] + " "

tok = learn(dataset_train["text"], 1000)   
l_BP = segment(words, tok)
l_wt = word_tokenize(words)
l_wp = wordpunct_tokenize(words)
list(WhitespaceTokenizer().span_tokenize(words))
print("My BPE tokenizer broke the test data into " + str(len(l_BP)) + " tokens while the nltk word_tokenize method returned " + str(len(l_wt)) + " words")
print (" and the wordpunct_tokenize function returned " + str(len(l_wp)) + " words.")




**Output**: My BPE tokenizer broke the test data into 17,188,243 tokens while the nltk word_tokenize method returned 6,900,297 words
 and the wordpunct_tokenize function returned 6,941,137 words. The computation completed in 3h 38m 45s.

> The much larger number of tokens, 17.2 million vs 6.9, is because my tokens were limited to the top 1000. This means that most of the tokens are still word fragments, and will thus exceed the number of words in the testing data (which is 5.7 million). The other tokenizers also exceed this because they treat punctuation and other symbols as their own words, but are much closer to the 5.7 million words. 




> 





The nltk tokenizers seem much better at actually extracting real, meaningful, and useful words rather than the simple pairings that BPE generates first. I imagine this allows BPE to ensure that it can deal with words it has not seen in training in a way that I'm not sure the nltk tokenizers can do, but it seems that BPE has to 'brute-force' things a bit more.

---


When running BPE the first things I get are word fragments, like common ends of words or the ever-present "th" pairing. The nltk tokenizers give me a list of the actual words, since it appears they just break along whitespace. Does this make BPE less language specific? I imagine that, for languages that cannot be so easily broken along whitespace as english, BPE could still add significant value in breaking down the text.

> I was a bit unclear as to what the Whitespace_Tokenizer actually does since I'm unfamiliar with token-span. The output did not seem to correlate with the BPE or other nltk tokenizers. 

Another big difference was time. I am not sure if it is the algorithm, advanced implementation on nltk's end, or shoddy implementation on my end, but far more information is gleaned per unit of time from the built in tokenizers. To find the first 30 tokens using my implementation of BPE, which includes only the simplest words like "the" and "an", about a minute of computation was necessary. 

---

wordpunct_tokenize required only 2 seconds to complete, and returns what would immediately appear to be more useful data. word_tokenize needed 43 seconds, excruciating when compared to wordpunct_tokenize but far faster, with far more useful words returned, than my BPE tokenizer implementation. The overlap between the BPE tokenizer and the word(punct)_tokenizer is minimal, including only the shortest of common stop words. The vast majority of the tokens returned by BPE are 2-4 character combinations that make up words.











I believe BPE could likely be enhanced in a couple of ways. For one, there is likely room to optimize my own implementation. 
However, beyond that, I believe the other nltk tokenizers see much improved performance for a few reasons

1.   **Whitespace as a useful separator**. While I'm not sure how unseen words are managed on these tokenizers, the immediate return of words, rather than tokens built from the ground up by the BPE tokenizer, seems far more efficient and is my guess as to why better performance is seen in less time
2.   **Better punctuation management**. The nltk tokenizers have punctuation broken out usefully into separate words. BPE only associates punctuation with previous characters rather than distinct features of their own. Breaking these symbols out into their own separate representation would likely improve utility.
3. **Top-down approach**. I wonder if tokenizer algorithms could be enhanced by only performning BPE on the text that is 'left over', or not recnignized, after extracting known words and punctuation immediately. This seems like it would eliminate the amount of text that would have to be built from the bottom-up, and the text that is immediately recognized as words or symbols by basic white-space processing could be identified faster

 